In [13]:
import pandas as pd
import json
import requests

In [14]:
from API_keys import myBEAAPI
from county_codes import stco_fips

In [15]:
base_url = f'https://apps.bea.gov/api/data/?UserID={myBEAAPI}'

### User variables

In [16]:
#Start & end years
year1 = '2018'
year0 = '2008'
years = f'{year0},{year1}'

#Nominal GDP for total GDP in 2018
nomGDP = 'CAGDP1'
nomGDP_lc = '3'

#Real GDP - 2012-chained GDP for change calcs
realGDP = 'CAGDP9'
realGDP_lc = '1'

geo = 'COUNTY'

#### Get Parameter Values for Regional Dataset (table IDs)

In [17]:
#Get data from specific table
def get_data(tablename,linecode,geofips,year):    
    url = f'{base_url}&method=GetData&datasetname=Regional&TableName={tablename}&LineCode={linecode}&GeoFIPS={geofips}&Year={year}'
    resp = requests.get(url).json()
    df = pd.DataFrame(resp['BEAAPI']['Results']['Data'])
    df = df[df['GeoFips'].isin(stco_fips)]
    return df

def var_cal(df):
    df['GDPrY0Y1'] = df[year1] - df[year0]
    df['GDPprY0Y1'] = ((df[year1]/df[year0])**(1/(int(year1)-int(year0))))-1
    return df

def clean_table(df):
    dff = df.drop(columns=[year0,year1]).rename(columns={'DataValue':'GDPY1'})
    for column_name in dff:
        dff.rename(columns={column_name:column_name.replace('Y0',year0[2:]).replace('Y1',year1[2:])},inplace=True)
    return dff

### County Table

In [18]:
#Get nominal GDP for Year 1 (Current Year)
df_nom = get_data(nomGDP,nomGDP_lc,geo,year1)

In [19]:
#Get real GDP to calculate change (Current - Init Year)
df_real = get_data(realGDP,realGDP_lc,geo,years)
df_real = df_real.pivot(index='GeoFips',columns='TimePeriod',values='DataValue')

In [20]:
#Merge nominal and real gdp
gdp = pd.merge(df_real,df_nom,how='left',on='GeoFips').drop(columns=['CL_UNIT','Code','NoteRef','TimePeriod','GeoName','UNIT_MULT'],axis=1)

In [21]:
#Clean data,
col = [year0,year1,'DataValue']
for i in col:
    gdp[i] = gdp[i].str.replace(',','').astype(float)
    gdp[i] = gdp[i]*1000

In [22]:
gdp = var_cal(gdp)

In [23]:
gdp

,GeoFips,2008,2018,DataValue,GDPrY0Y1,GDPprY0Y1
0,09001,8.992901e+10,7.795124e+10,8.938766e+10,-1.197778e+10,-0.014192
1,09005,8.039525e+09,7.688830e+09,8.742282e+09,-3.506950e+08,-0.004450
2,09009,4.876885e+10,4.806027e+10,5.361390e+10,-7.085790e+08,-0.001463
3,34003,6.530798e+10,6.614889e+10,7.354832e+10,8.409130e+08,0.001280
4,34013,5.224085e+10,5.231117e+10,5.901832e+10,7.031900e+07,0.000135
5,34017,3.693236e+10,4.469040e+10,5.157592e+10,7.758039e+09,0.019250
6,34019,7.113885e+09,6.811313e+09,7.590001e+09,-3.025720e+08,-0.004337
7,34021,2.735990e+10,2.845950e+10,3.209390e+10,1.099599e+09,0.003948
8,34023,5.450641e+10,5.831963e+10,6.468701e+10,3.813218e+09,0.006785
9,34025,3.184466e+10,3.195471e+10,3.582647e+10,1.100490e+08,0.000345


### Intermediate County Table - GDP

In [24]:
co_gdp = clean_table(gdp)

In [25]:
co_gdp = co_gdp.rename(columns={'GeoFips':'GEO_ID'})
co_gdp

,GEO_ID,GDP18,GDPr0818,GDPpr0818
0,09001,8.938766e+10,-1.197778e+10,-0.014192
1,09005,8.742282e+09,-3.506950e+08,-0.004450
2,09009,5.361390e+10,-7.085790e+08,-0.001463
3,34003,7.354832e+10,8.409130e+08,0.001280
4,34013,5.901832e+10,7.031900e+07,0.000135
5,34017,5.157592e+10,7.758039e+09,0.019250
6,34019,7.590001e+09,-3.025720e+08,-0.004337
7,34021,3.209390e+10,1.099599e+09,0.003948
8,34023,6.468701e+10,3.813218e+09,0.006785
9,34025,3.582647e+10,1.100490e+08,0.000345


In [26]:
#to csv
#co_gdp.to_csv('../output/co_gdp.csv')

### Make Subregion and Region Tables

In [27]:
#Read in geography cross-walk file & clean id column for join
geo_xwalk = pd.read_excel('../data/31CR_CoxSub.xlsx')
geo_xwalk['stco'] = geo_xwalk['stco'].apply(lambda x: '{0:0>5}'.format(x))
geo_xwalk.head()

,stco,st,co,stco_int,reg,subreg,stco_lbl,co_lbl
0,09001,9,1,9001,31CR,CT,"Fairfield County, Connecticut",Fairfield
1,09005,9,5,9005,31CR,CT,"Litchfield County, Connecticut",Litchfield
2,09009,9,9,9009,31CR,CT,"New Haven County, Connecticut",New Haven
3,34003,34,3,34003,31CR,INJ,"Bergen County, New Jersey",Bergen
4,34013,34,13,34013,31CR,INJ,"Essex County, New Jersey",Essex


In [28]:
#Merge xwalk file to reduce to region counties only & drop unneeded identifiers
df = pd.merge(gdp,geo_xwalk,how='left',left_on=['GeoFips'],right_on=['stco']).drop(columns=['st','co','stco_int','stco','stco_lbl','co_lbl','GDPrY0Y1','GDPprY0Y1'])
df.head()

,GeoFips,2008,2018,DataValue,reg,subreg
0,09001,8.992901e+10,7.795124e+10,8.938766e+10,31CR,CT
1,09005,8.039525e+09,7.688830e+09,8.742282e+09,31CR,CT
2,09009,4.876885e+10,4.806027e+10,5.361390e+10,31CR,CT
3,34003,6.530798e+10,6.614889e+10,7.354832e+10,31CR,INJ
4,34013,5.224085e+10,5.231117e+10,5.901832e+10,31CR,INJ


#### Subregion

In [29]:
sub_gdp = df.drop(columns=['reg','GeoFips']).groupby(['subreg']).sum()

In [31]:
sub_gdp = var_cal(sub_gdp)
sub_gdp = clean_table(sub_gdp)
sub_gdp

,GDP18,GDPr0818,GDPpr0818
subreg,,,
CT,1.517438e+11,-1.303705e+10,-0.009261
INJ,4.117786e+11,2.310904e+10,0.006489
LI,1.863085e+11,1.254012e+10,0.008068
LowHud,1.098027e+11,1.286486e+10,0.014665
MidHud,4.170134e+10,2.814753e+09,0.007978
NYC,9.930176e+11,2.101598e+11,0.029117
ONJ,1.067618e+11,2.207989e+09,0.002361


In [ ]:
#to csv
#sub_gdp.to_csv('../output/sub_gdp.csv')

#### Region

In [32]:
reg_gdp = df.drop(columns=['subreg','GeoFips']).groupby(['reg']).sum()

In [33]:
reg_gdp = var_cal(reg_gdp)
reg_gdp = clean_table(reg_gdp)
reg_gdp

,GDP18,GDPr0818,GDPpr0818
reg,,,
31CR,2.001114e+12,2.506595e+11,0.015736


In [ ]:
#to csv
#reg_gdp.to_csv('../output/reg_gdp.csv')